# 🔒 From Discovery to Lockdown
### Automating tagging of sensitive columns in BigQuery

> **Authors**: Bharat Tiwari, Varun Joshi, Vignesh Rajamani  
> **Last Updated**: Sep 8, 2025


## 1. Introduction
As a data security administrator, protecting your organization's data is paramount. Manual classification is risky and inefficient.

This notebook implements a **modern, automated approach**:
1.  **Discovery**: Using **Sensitive Data Protection (SDP)** to find sensitive data.
2.  **Enforcement**: Using **IAM Data Governance Tags** to secure it.

This "Glue Code" solution bridges the gap, automating the application of tags based on discovery results.


## 2. Discovery with Sensitive Data Protection (SDP)
**Goal**: Identify where your sensitive data resides.

### 📋 Steps to Configure Scan
1.  Go to **Sensitive Data Protection > Discovery > Scan Configurations**.
2.  Click **Create configuration**.
3.  Select **BigQuery** as the data source.
4.  Define your **Scan Scope** (Org, Project, or Table).
5.  Select your **Inspection Template** (InfoTypes to scan for).

<div style="background-color: #e8f0fe; border-left: 5px solid #4285f4; padding: 10px; margin: 10px 0;">
    <strong>💡 Critical Step: Save to BigQuery</strong><br>
    In the "Actions" section, you MUST select <strong>Save data profile to BigQuery</strong>.
    <br><br>
    <ul>
        <li>Let the service create the table for you to ensure the schema is correct.</li>
        <li><strong>Note down</strong> the <code>Project ID</code>, <code>Dataset ID</code>, and <code>Table ID</code> of this results table.</li>
    </ul>
</div>

#### Schema Overview
The results table (`discovery_profiles`) will contain the sensitivity scores and location of each column. We will query this table in the automation step.


## 3. Define IAM Data Governance Tags
**Goal**: Create the labels that will be attached to your columns.

### 🏷️ Create Your Tags
1.  Navigate to **IAM & Admin > Tags**.
2.  Click **Create**.
3.  Define your Key and Values.

**Example Structure:**
*   **Key**: `data_sensitivity`
*   **Values**: 
    *   `High`
    *   `Medium`
    *   `Low`

We will map these values to the SDP sensitivity scores in the configuration step below.


## 4. Run Automation Logic
**Goal**: Read SDP results and apply IAM Tags automatically.

Run the cells below to execute the logic.


In [ ]:
# -*- coding: utf-8 -*-
from google.cloud import bigquery
from datetime import datetime

# Initialize Client
client = bigquery.Client()
print(f"✅ Client initialized at {datetime.now()}")

In [ ]:
# ---------------------------------------------------------
# CONFIGURATION
# ---------------------------------------------------------

# Option 1: Interactive Input (Run this cell to be prompted)
try:
    print("--- 1. SOURCE (SDP Results) ---")
    project_id = input("Enter Project ID of SDP Profile Table: ")
    sdp_dataset_id = input("Enter Dataset ID of SDP Profile Table: ")
    sdp_table_id = input("Enter Table ID of SDP Profile Table: ")
    
    full_table_id = f"{project_id}.{sdp_dataset_id}.{sdp_table_id}"
    print(f"📖 Reading from: {full_table_id}")

    print("\n--- 2. DESTINATION (Tagging Log) ---")
    destination_dataset_id = input("Enter Dataset ID for Log Table: ")
    destination_table_id = input("Enter Table ID for Log Table: ")
    full_destination_table_id = f"{project_id}.{destination_dataset_id}.{destination_table_id}"
    print(f"📝 Logging to: {full_destination_table_id}")

except Exception as e:
    print(f"❌ Error reading inputs: {e}")

In [ ]:
# ---------------------------------------------------------
# TAG MAPPING
# ---------------------------------------------------------
tag_config = {}

def get_tag_input(sensitivity):
    print(f"\nFor Sensitivity: '{sensitivity}'")
    tag_key = input(f"  > Enter Full Tag Key (e.g. prompt-project/sensitivity): ")
    tag_value = input(f"  > Enter Tag Value (e.g. High): ")
    return tag_key, tag_value

print("--- 3. DEFINE TAG RULES ---")
try:
    # You can comment these out if you don't use specific levels
    # Defaulting to prompt for High, Moderate, Low
    tag_config["SENSITIVITY_HIGH"] = get_tag_input("SENSITIVITY_HIGH")
    tag_config["SENSITIVITY_MODERATE"] = get_tag_input("SENSITIVITY_MODERATE")
    tag_config["SENSITIVITY_LOW"] = get_tag_input("SENSITIVITY_LOW")
    
    print("\n✅ Configuration complete.")
    print(f"Rules: {json.dumps(tag_config, indent=2)}")
except Exception as e:
    print(f"❌ Error reading tags: {e}")

In [ ]:
# ---------------------------------------------------------
# EXECUTION ENGINE
# ---------------------------------------------------------

output_rows = []

sql_query = f"""
SELECT
    column_profile.dataset_project_id,
    column_profile.dataset_id,
    column_profile.table_id,
    column_profile.column,
    column_profile.column_type,
    column_profile.sensitivity_score.score
FROM
    `{full_table_id}`
WHERE 1=1
    AND _PARTITIONTIME = (SELECT MAX(_partitiontime) FROM `{full_table_id}`)
    AND column_profile.column IS NOT NULL
    AND column_profile.table_id <> '{sdp_table_id}'
"""

print("⏳ Running Discovery Query...")

try:
    query_job = client.query(sql_query)
    results_list = list(query_job.result())
    print(f"✅ Found {len(results_list)} columns to analyze.")

    for row in results_list:
        dataset_project = row.dataset_project_id
        dataset = row.dataset_id
        table = row.table_id
        column = row.column
        sensitivity = row[5]
        
        applied_key, applied_val = None, None
        
        if sensitivity in tag_config:
            tag_key, tag_value = tag_config[sensitivity]
            applied_key, applied_val = tag_key, tag_value
            
            print(f"\n🔹 MATCH: {dataset}.{table}.{column} ({sensitivity})")
            print(f"   Action: Apply {tag_value}")

            alter_stmt = f"""
            ALTER TABLE `{dataset_project}.{dataset}.{table}`
            ALTER COLUMN `{column}`
            SET OPTIONS (
              data_governance_tags = [('{tag_key}', '{tag_value}')]
            );
            """
            
            # EXECUTE
            try:
                # update_job = client.query(alter_stmt)
                # update_job.result()
                print(f"   ✅ [DRY RUN] Tag Applied (Simulated)")
                
                # UNCOMMENT BELOW TO ENABLE REAL TAGGING
                update_job = client.query(alter_stmt)
                update_job.result()
                print(f"   ✅ Tag Successfully Applied!")
                
            except Exception as e:
                print(f"   ❌ Failed to apply tag: {e}")
        
        else:
            pass

        output_rows.append((dataset_project, dataset, table, column, row.column_type, sensitivity, applied_key, applied_val))

    # SAVE LOGS
    print(f"\n💾 Saving audit logs to {full_destination_table_id}...")
    schema = [
        bigquery.SchemaField("dataset_project_id", "STRING"),
        bigquery.SchemaField("dataset_id", "STRING"),
        bigquery.SchemaField("table_id", "STRING"),
        bigquery.SchemaField("column", "STRING"),
        bigquery.SchemaField("column_type", "STRING"),
        bigquery.SchemaField("sensitivity_score", "STRING"),
        bigquery.SchemaField("tag_key", "STRING"),
        bigquery.SchemaField("tag_value", "STRING"),
    ]
    
    table_ref = bigquery.Table(f"{project_id}.{destination_dataset_id}.{destination_table_id}", schema=schema)
    client.create_table(table_ref, exists_ok=True)
    errors = client.insert_rows(table_ref, output_rows)
    
    if not errors:
        print("✅ Audit log saved successfully.")
    else:
        print(f"⚠️ Errors saving audit log: {errors}")

except Exception as e:
    print(f"❌ Fatal Error: {e}")


## 5. Conclusion
✅ **Done!**
Your Sensitive Data Protection findings have been translated into IAM Data Governance Tags. Access policies attached to these tags are now actively protecting your data.
